In [1]:
from pyspark import SparkContext , SparkConf
from pyspark.sql import SparkSession
import numpy as np
sc = SparkContext.getOrCreate()

In [8]:
trainPath="../train.csv"
predictionPath="../../../Downloads/predictionsCollViaContentNoSeen.csv"

In [9]:
def ap(RankedList, PositiveItems, at=None):
    """
    Calculates AP@_ 
    """
    if(len(PositiveItems)==0):
        return 0
    print("Recomended: ", RankedList)
    RankedList = RankedList[:at]
    is_relevant = np.in1d(RankedList, PositiveItems, assume_unique=True)
    print("Positives: ", PositiveItems)
    print("is_relevant: ", is_relevant)   
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(len(is_relevant)))
    print("P at k :" , p_at_k)
    map_score = np.sum(p_at_k) / np.min([len(PositiveItems), len(RankedList)])
    print(map_score)
    assert 0 <= map_score <= 1, map_score
    
    return map_score

In [12]:
#compute relevants
trainSet = sc.textFile(trainPath)
trainSet = trainSet.map(lambda l: l.split(','))
trainSet = trainSet.filter(lambda line: 'userId' not in line)
trainSet = trainSet.map(lambda line: (int(line[0]), int(line[1]), float(line[2])))
relevantsPerUser = trainSet.filter(lambda line: line[2] >= 8)\
                            .map(lambda line: (line[0], [line[1]]))\
                            .reduceByKey(lambda x, y: x + y)
relevantsDict = relevantsPerUser.collectAsMap()

In [13]:
#compute recsRDD

recomendations = sc.textFile(predictionPath)
recomendations = recomendations.map(lambda l: l.split(','))
recomendations = recomendations.filter(lambda l: 'userId' not in l)
recomendations = recomendations.map(lambda x: (int(x[0]),x[1].split(" ")[:5]))
recomendations.take(5)

[(4, ['25656', '23217', '32578', '17380', '28102']),
 (5, ['17815', '2164', '22211', '15743', '33755']),
 (8, ['13473', '25421', '20407', '28102', '36853']),
 (9, ['3674', '31444', '30287', '21867', '34390']),
 (13, ['9855', '15469', '5861', '23137', '22870'])]

In [14]:
recomendations = recomendations.map(lambda x: (x[0],list(map(lambda y: int(y),x[1]))))
recomendations.take(5)

[(4, [25656, 23217, 32578, 17380, 28102]),
 (5, [17815, 2164, 22211, 15743, 33755]),
 (8, [13473, 25421, 20407, 28102, 36853]),
 (9, [3674, 31444, 30287, 21867, 34390]),
 (13, [9855, 15469, 5861, 23137, 22870])]

In [15]:
MAP = recomendations.map(lambda x: ap(x[1], relevantsDict.get(x[0],[]), at=5))
MapV = MAP.sum() / MAP.count()
print("MAP@5: {}".format(MapV))

MAP@5: 0.5305115983476324
